# Linear Regression Consulting Project
Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark
findspark.init('/home/fernando/spark-2.4.6-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('consulting').getOrCreate()

from pyspark.ml.regression import LinearRegression

In [2]:
df = spark.read.csv('Linear_Regression/cruise_ship_info.csv', inferSchema=True, header=True)

In [3]:
df.count()

158

In [4]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [7]:
df.show(3)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 3 rows



In [6]:
for ship in df.head(3):
    print(ship,'\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55) 

Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55) 

Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7) 



# String Indexer

#### Cruise_line is a string, and may be useful for the model, we need to convert it to a sort of numeric value than the model can understand

In [17]:
df.groupBy('Cruise_line').count().orderBy('count').show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|           Orient|    1|
|          Crystal|    2|
|           Disney|    2|
|          Azamara|    2|
|         Seabourn|    3|
|          Oceania|    3|
|         Windstar|    3|
|           Cunard|    3|
|        Silversea|    4|
|Regent_Seven_Seas|    5|
|              P&O|    6|
|             Star|    6|
|              MSC|    8|
|        Celebrity|   10|
|            Costa|   11|
|        Norwegian|   13|
| Holland_American|   14|
|         Princess|   17|
|         Carnival|   22|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [18]:
from pyspark.ml.feature import StringIndexer

In [19]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='cruise_category')
indexed = indexer.fit(df).transform(df)

In [21]:
indexed.show(10)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_category|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|

# Generating features for the model
# Target: Crew

#### Vectors and assembler object

In [22]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [23]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_category']

In [28]:
assembler = VectorAssembler(inputCols = [
                                'Age',
                                'Tonnage',
                                'passengers',
                                'length',
                                'cabins',
                                'passenger_density',
                                'cruise_category'
                            ], outputCol='features')

In [29]:
output = assembler.transform(indexed)

In [35]:
output.select('Ship_name','features', 'crew').show(10)

+-----------+--------------------+----+
|  Ship_name|            features|crew|
+-----------+--------------------+----+
|    Journey|[6.0,30.276999999...|3.55|
|      Quest|[6.0,30.276999999...|3.55|
|Celebration|[26.0,47.262,14.8...| 6.7|
|   Conquest|[11.0,110.0,29.74...|19.1|
|    Destiny|[17.0,101.353,26....|10.0|
|    Ecstasy|[22.0,70.367,20.5...| 9.2|
|    Elation|[15.0,70.367,20.5...| 9.2|
|    Fantasy|[23.0,70.367,20.5...| 9.2|
|Fascination|[19.0,70.367,20.5...| 9.2|
|    Freedom|[6.0,110.23899999...|11.5|
+-----------+--------------------+----+
only showing top 10 rows



## Get final data for the model

In [36]:
final_data = output.select('features', 'crew')

In [37]:
final_data.show(10)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
+--------------------+----+
only showing top 10 rows



## Train test split

In [38]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [39]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               109|
|   mean| 7.611284403669733|
| stddev|3.5243481406515094|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [40]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                49|
|   mean| 8.201020408163265|
| stddev|3.4576480716051528|
|    min|               1.6|
|    max|              19.1|
+-------+------------------+



# Linear Regression Model

In [41]:
from pyspark.ml.regression import LinearRegression

In [42]:
ship_lr = LinearRegression(featuresCol='features', labelCol='crew')

In [43]:
trained_ship_model = ship_lr.fit(train_data)

In [45]:
ship_results = trained_ship_model.evaluate(test_data)

In [48]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               109|
|   mean| 7.611284403669733|
| stddev|3.5243481406515094|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [47]:
ship_results.rootMeanSquaredError

1.3566238677708025

In [49]:
ship_results.r2

0.8428507999491135

In [50]:
ship_results.meanSquaredError

1.840428318605412

In [51]:
ship_results.meanAbsoluteError

0.7407303583978361

# Check results

In [52]:
from  pyspark.sql.functions import corr

In [53]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

Are the features correlated with the crew?

In [54]:
df.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [55]:
df.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

